In [27]:
import pandas as pd
pd.set_option("display.max_rows",205)

### Data Imports

Medicaid provider listing and taxonomy from CMS for psychiatrists, mental health services and psychologists

In [13]:
dtypes = {"NPI":'str'}
df1 = pd.read_csv("Medicaid_Enrolled_Provider_Listing.csv", dtype=dtypes)

/Users/shelbyjouppi/.pyenv/versions/3.9.7/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [63]:
dtypes = {"NPI":'str'}
df2 = pd.read_csv("cmi-scrape-2.csv", dtype=dtypes)

### Data Cleaning

In [14]:
# renaming columns
df1.columns = df1.columns.str.lower().str.replace(" ","_")

In [64]:
df2.columns = df2.columns.str.lower().str.replace(" ","_")

In [15]:
df1.head()

,medicaid_provider_id,npi,provider_or_facility_name,medicaid_type,profession_or_service,provider_specialty,service_address,city,state,zip_code,county,telephone,latitude,longitude,enrollment_begin_date,next_anticipated_revalidation_date,file_date,medically_fragile_children_directory_ind,provider_email
0,13098,1477672988,JCCA FH DIVISION,FFS,CHILD (FOSTER) CARE AGENCIES,NaN,1075 BROADWAY,PLEASANTVILLE,NY,10570-2345,WESTCHESTER,NaN,41.12940,-73.77794,02/01/1979,12/11/2020,02/28/2022,N,NaN
1,13098,1477672988,JCCA FH DIVISION,FFS,CHILD (FOSTER) CARE AGENCIES,NaN,111 N 3RD AVE VERNONDALE RES,MOUNT VERNON,NY,10550-1364,WESTCHESTER,NaN,40.91451,-73.83634,02/01/1979,12/11/2020,02/28/2022,N,NaN
2,13098,1477672988,JCCA FH DIVISION,FFS,CHILD (FOSTER) CARE AGENCIES,NaN,162 19 76TH AVE,FLUSHING,NY,11366-1133,QUEENS,NaN,40.72670,-73.79774,02/01/1979,12/11/2020,02/28/2022,N,NaN
3,13098,1477672988,JCCA FH DIVISION,FFS,CHILD (FOSTER) CARE AGENCIES,NaN,16208 77TH AVE,FLUSHING,NY,11366-1022,QUEENS,NaN,40.72416,-73.80730,02/01/1979,12/11/2020,02/28/2022,N,NaN
4,13098,1477672988,JCCA FH DIVISION,FFS,CHILD (FOSTER) CARE AGENCIES,NaN,1810 AVENUE H,BROOKLYN,NY,11230-1820,KINGS,NaN,40.63024,-73.95893,02/01/1979,12/11/2020,02/28/2022,N,NaN


In [16]:
df2.head()

,npi,primary_taxonomy,selected_taxonomy,state,license_number
0,1689635815,Yes,2084P0800X - Psychiatry & Neurology Psychiatry,NY,104064
1,1689635815,No,2084P0804X - Psychiatry & Neurology Child & A...,NY,104064
2,1164462974,Yes,2084N0400X - Psychiatry & Neurology Neurology,NY,106944
3,1508932989,Yes,2084P0800X - Psychiatry & Neurology Psychiatry,NY,107772
4,1366430290,Yes,2084P0800X - Psychiatry & Neurology Psychiatry,NY,108074-1


In [65]:
# Isolating weird formatting from the scrape
df2[df2.selected_taxonomy.str.contains("\n")].selected_taxonomy.value_counts()

193400000X SINGLE SPECIALTY  GROUP\n\t\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t363LP0808X  - Nurse Practitioner Psych/Mental Health                              32
193200000X MULTI-SPECIALTY GROUP\n\t\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t1041C0700X  - Social Worker Clinical                                                26
193200000X MULTI-SPECIALTY GROUP\n\t\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t104100000X  - Social Worker                                                         21
193200000X MULTI-SPECIALTY GROUP\n\t\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t101YM0800X  - Counselor Mental Health                                               17
193200000X MULTI-SPECIALTY GROUP\n\t\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t363LP0808X  - Nurse Practitioner Psych/Mental Health                                17
193200000X MULTI-SPECIALTY GROUP\n\t\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t103T00000X  - Psychologist                                                          16
193400000X SINGLE SPECIALTY  GROUP

In [66]:
# Cleaning up weird formatting from the scrape
df2.selected_taxonomy = df2.selected_taxonomy \
.str.replace("193400000X SINGLE SPECIALTY  GROUP\n\t\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t","") \
.str.replace("193200000X MULTI-SPECIALTY GROUP\n\t\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t","") \
.str.replace("193400000X MULTIPLE SINGLE SPECIALTY GROUP\n\t\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t","")

In [67]:
# Looks much better
df2.selected_taxonomy.value_counts()

103TC0700X  - Psychologist Clinical                                                                          2467
103T00000X  - Psychologist                                                                                   1649
363LP0808X  - Nurse Practitioner Psych/Mental Health                                                         1623
2084P0800X  - Psychiatry & Neurology Psychiatry                                                               523
2084P0804X  - Psychiatry & Neurology Child & Adolescent Psychiatry                                            435
103TC2200X  - Psychologist Clinical Child & Adolescent                                                        291
163W00000X  - Registered Nurse                                                                                271
103G00000X  - Clinical Neuropsychologist                                                                      257
390200000X  - Student in an Organized Health Care Education/Training Program            

In [68]:
### I'm only interested in providers licensed in NY
df2 = df2.query('state == "NY"')

In [37]:
### I'm only interested in providers listed as having the following services or specialies
specialty = ['CHILD PSYCHIATRY', 'MENTAL HLTH SVCS CHILD']
service = ['CLINICAL PSYCHOLOGIST']
# sw = ['CLINICAL SOCIAL WORKER']

In [38]:
mental_health = df1[(df1.profession_or_service.isin(service)) | (df1.provider_specialty.isin(specialty))]

### Inspect the Data

In [69]:
### How many taxonomies listed?
df2.selected_taxonomy.value_counts().to_frame()

,selected_taxonomy
103TC0700X - Psychologist Clinical,2202
363LP0808X - Nurse Practitioner Psych/Mental Health,1423
103T00000X - Psychologist,1330
2084P0800X - Psychiatry & Neurology Psychiatry,391
2084P0804X - Psychiatry & Neurology Child & Adolescent Psychiatry,332
163W00000X - Registered Nurse,246
103TC2200X - Psychologist Clinical Child & Adolescent,223
103G00000X - Clinical Neuropsychologist,209
103TB0200X - Psychologist Cognitive & Behavioral,104
103TC1900X - Psychologist Counseling,97


In [ ]:
# Interesting taxonomies to look further into the providers
# Potentially Weave Into Analysis

# ------- 
# 208000000X - Pediatrics (11)
# Behavioral Analyst (11)
# 103TF0000X - Psychologist Family (46)
# 251S00000X - Community/Behavioral Health (54)
# 106H00000X - Marriage & Family Therapist 	10
# 3245S0500X - Substance Abuse Rehabilitation Facility Substance Abuse Treatment, Children 1
# 364SP0807X - Clinical Nurse Specialist Psych/Mental Health, Child & Adolescent 	2
# 163WP0807X - Registered Nurse Psych/Mental Health, Child & Adolescent 	8
# 261QM0855X - Clinic/Center Adolescent and Children Mental Health 	29

In [36]:
### providers in the 'MENTAL HLTH SVCS CHILD' category
df1.query("provider_specialty =='MENTAL HLTH SVCS CHILD'").provider_or_facility_name.value_counts().to_frame().head(40)

,provider_or_facility_name
UNITY HOSPITAL ROCHESTER,207
STRONG MEMORIAL HOSPITAL,185
ROCHESTER GENERAL HOSPITAL,166
NYS ARC (CHAUTAUQUA CNTY),155
MARY IMOGENE BASSETT HOSPITAL,125
NYU LANGONE HOSPITALS,116
BROOKHAVEN MEMORIAL HOSPITAL,99
BROOKDALE HOSPITAL MEDICAL CENTER,96
MOUNT SINAI HOSPITAL,95
NEW YORK PRESBYTERIAN HOSPITAL,94


In [39]:
### County breakdown of services
mental_health.county.value_counts()

NEW YORK          2630
KINGS             1831
SUFFOLK           1536
QUEENS            1428
BRONX             1345
NASSAU            1204
MONROE            1196
ERIE              1120
WESTCHESTER        979
ONONDAGA           639
ALBANY             488
RICHMOND           461
DUTCHESS           352
OUT OF STATE       344
CHAUTAUQUA         288
ORANGE             285
ROCKLAND           268
SARATOGA           226
ULSTER             212
BROOME             179
RENSSELAER         157
NIAGARA            153
JEFFERSON          141
SCHENECTADY        132
ONEIDA             129
OSWEGO             119
WARREN             115
CLINTON            114
OTSEGO             109
ONTARIO             94
PUTNAM              93
CHEMUNG             88
WAYNE               80
COLUMBIA            74
TOMPKINS            72
SULLIVAN            69
STEUBEN             66
MONTGOMERY          61
CATTARAUGUS         61
SAINT LAWRENCE      58
GENESEE             56
DELAWARE            54
CORTLAND            54
WASHINGTON 

### Merge the two datasets

In [84]:
# For simplification purposes lets look at providers by their primary taxonomies
df2_primary = df2.query("primary_taxonomy == 'Yes'")

In [88]:
df = mental_health.merge(df2_primary, how="left",left_on="npi", right_on="npi")

### How many mental health providers specialize in children?
Let's just look at the providers that declare a primary taxonomy mentioning "child" or "pediatric"

In [58]:
# Pediaitrics is yielding medical related taxonomies, so I'm just going to stick with 'child'
searchfor = ['child', 'pediatric']
df2[df2.selected_taxonomy.str.contains('|'.join(searchfor),na=False,case=False)].selected_taxonomy.value_counts()

2084P0804X  - Psychiatry & Neurology Child & Adolescent Psychiatry                           332
103TC2200X  - Psychologist Clinical Child & Adolescent                                       223
261QM0855X  - Clinic/Center Adolescent and Children Mental Health                             29
208000000X  - Pediatrics                                                                      12
163WP0807X  - Registered Nurse Psych/Mental Health, Child & Adolescent                         8
363LP0200X  - Nurse Practitioner Pediatrics                                                    7
364SP0810X  - Clinical Nurse Specialist Psych/Mental Health, Child & Family                    5
364SP0807X  - Clinical Nurse Specialist Psych/Mental Health, Child & Adolescent                2
2080P0006X  - Pediatrics Developmental - Behavioral Pediatrics                                 1
2080A0000X  - Pediatrics Adolescent Medicine                                                   1
3245S0500X  - Substance Abuse 

In [79]:
df2.query("selected_taxonomy.str.contains(('child'), case=False, na=False)").selected_taxonomy.value_counts()

2084P0804X  - Psychiatry & Neurology Child & Adolescent Psychiatry                           332
103TC2200X  - Psychologist Clinical Child & Adolescent                                       223
261QM0855X  - Clinic/Center Adolescent and Children Mental Health                             29
163WP0807X  - Registered Nurse Psych/Mental Health, Child & Adolescent                         8
364SP0810X  - Clinical Nurse Specialist Psych/Mental Health, Child & Family                    5
364SP0807X  - Clinical Nurse Specialist Psych/Mental Health, Child & Adolescent                2
3245S0500X  - Substance Abuse Rehabilitation Facility Substance Abuse Treatment, Children      1
282NC2000X  - General Acute Care Hospital Children                                             1
Name: selected_taxonomy, dtype: int64

In [77]:
# How does that look when we look at just primary specialty
df2[(df2.selected_taxonomy.str.contains('child',case=False, na=False)) & (df2.primary_taxonomy =='Yes')].selected_taxonomy.value_counts()

2084P0804X  - Psychiatry & Neurology Child & Adolescent Psychiatry                           231
103TC2200X  - Psychologist Clinical Child & Adolescent                                       111
261QM0855X  - Clinic/Center Adolescent and Children Mental Health                              7
364SP0810X  - Clinical Nurse Specialist Psych/Mental Health, Child & Family                    3
3245S0500X  - Substance Abuse Rehabilitation Facility Substance Abuse Treatment, Children      1
282NC2000X  - General Acute Care Hospital Children                                             1
163WP0807X  - Registered Nurse Psych/Mental Health, Child & Adolescent                         1
Name: selected_taxonomy, dtype: int64

In [61]:
# What if we just look at no duplicates
df2[(df2.selected_taxonomy.str.contains('child',case=False, na=False))].drop_duplicates(subset="npi").selected_taxonomy.value_counts()

2084P0804X  - Psychiatry & Neurology Child & Adolescent Psychiatry                           330
103TC2200X  - Psychologist Clinical Child & Adolescent                                       221
261QM0855X  - Clinic/Center Adolescent and Children Mental Health                             27
163WP0807X  - Registered Nurse Psych/Mental Health, Child & Adolescent                         8
364SP0810X  - Clinical Nurse Specialist Psych/Mental Health, Child & Family                    5
364SP0807X  - Clinical Nurse Specialist Psych/Mental Health, Child & Adolescent                2
3245S0500X  - Substance Abuse Rehabilitation Facility Substance Abuse Treatment, Children      1
282NC2000X  - General Acute Care Hospital Children                                             1
Name: selected_taxonomy, dtype: int64

In [73]:
df2[(df2.selected_taxonomy.str.contains('child',case=False, na=False))].npi.value_counts().head(50)

1508929621    3
1407980766    2
1811185440    2
1578641825    2
1720246549    2
1093959546    1
1538298914    1
1003962267    1
1417056136    1
1992893382    1
1891702361    1
1174619639    1
1477533420    1
1134281777    1
1396964094    1
1730211277    1
1376696229    1
1346454279    1
1912069154    1
1699877803    1
1679670335    1
1255447751    1
1003874207    1
1518030774    1
1962496463    1
1417026717    1
1255482766    1
1215087010    1
1164548079    1
1346256591    1
1578613287    1
1205806981    1
1265598338    1
1346362951    1
1659421055    1
1295844280    1
1740393735    1
1548233604    1
1194949610    1
1477632297    1
1245351816    1
1992838874    1
1851385546    1
1730251968    1
1518080878    1
1366583494    1
1770578197    1
1437113412    1
1619952694    1
1851457790    1
Name: npi, dtype: int64

In [74]:
duplicates = ['1508929621','1407980766','1811185440','1578641825','1720246549']

In [75]:
df2[df2.npi.isin(duplicates)]

,npi,primary_taxonomy,selected_taxonomy,state,license_number
600,1578641825,Yes,2084P0804X - Psychiatry & Neurology Child & A...,NY,190284
601,1578641825,No,2084P0804X - Psychiatry & Neurology Child & A...,NY,190284-1
953,1407980766,Yes,2084P0804X - Psychiatry & Neurology Child & A...,NY,F400781
954,1407980766,No,207T00000X - Neurological Surgery,NY,15521
955,1407980766,No,2084P0800X - Psychiatry & Neurology Psychiatry,NY,F400781
956,1407980766,No,2084P0804X - Psychiatry & Neurology Child & A...,NY,400781
1288,1508929621,Yes,261QM0855X - Clinic/Center Adolescent and Chi...,NY,6795122A
1289,1508929621,No,261QM0855X - Clinic/Center Adolescent and Chi...,NY,6795123A
1290,1508929621,No,261QM0855X - Clinic/Center Adolescent and Chi...,NY,6795120A
1291,1508929621,No,261QM1300X - Clinic/Center Multi-Specialty,NY,080211497


In [80]:
primary_no = df2[(df2.selected_taxonomy.str.contains('child',case=False, na=False))&(df2.primary_taxonomy =='No')].npi.to_list()

In [82]:
# Let's look at those who don't have children as their primary taxonomy. What ARE their primary taxonomies
df2[df2.npi.isin(primary_no)].groupby("primary_taxonomy").selected_taxonomy.value_counts()

primary_taxonomy  selected_taxonomy                                                                   
No                103TC2200X  - Psychologist Clinical Child & Adolescent                                  112
                  2084P0804X  - Psychiatry & Neurology Child & Adolescent Psychiatry                      101
                  103TC0700X  - Psychologist Clinical                                                      44
                  103T00000X  - Psychologist                                                               43
                  103TF0000X  - Psychologist Family                                                        30
                  2084P0800X  - Psychiatry & Neurology Psychiatry                                          29
                  103TB0200X  - Psychologist Cognitive & Behavioral                                        28
                  261QM0855X  - Clinic/Center Adolescent and Children Mental Health                        22
                 

In [92]:
# Checks out
df.query("selected_taxonomy.str.contains(('child'), case=False, na=False)").drop_duplicates(subset='npi').selected_taxonomy.value_counts()

2084P0804X  - Psychiatry & Neurology Child & Adolescent Psychiatry                           231
103TC2200X  - Psychologist Clinical Child & Adolescent                                       111
261QM0855X  - Clinic/Center Adolescent and Children Mental Health                              7
364SP0810X  - Clinical Nurse Specialist Psych/Mental Health, Child & Family                    3
3245S0500X  - Substance Abuse Rehabilitation Facility Substance Abuse Treatment, Children      1
282NC2000X  - General Acute Care Hospital Children                                             1
163WP0807X  - Registered Nurse Psych/Mental Health, Child & Adolescent                         1
Name: selected_taxonomy, dtype: int64

In [93]:
df.query("selected_taxonomy.str.contains(('child'), case=False, na=False)").drop_duplicates(subset='npi').county.value_counts()

NEW YORK          68
SUFFOLK           42
BRONX             37
WESTCHESTER       29
KINGS             24
MONROE            22
NASSAU            22
ERIE              22
QUEENS            16
ALBANY            12
ONONDAGA          10
ORANGE             6
RICHMOND           5
ONTARIO            4
ROCKLAND           4
SARATOGA           3
OUT OF STATE       3
ULSTER             2
SAINT LAWRENCE     2
STEUBEN            2
OSWEGO             2
WARREN             2
DUTCHESS           2
SULLIVAN           2
TOMPKINS           2
PUTNAM             2
CHEMUNG            1
CORTLAND           1
SCHOHARIE          1
LIVINGSTON         1
SCHENECTADY        1
CATTARAUGUS        1
MADISON            1
ONEIDA             1
Name: county, dtype: int64

In [112]:
# Find providers with locations in muliple counties
# 
multiple_counties = df.query("selected_taxonomy.str.contains(('child'), case=False, na=False)").groupby('npi').county.value_counts().to_frame() \
.rename({'county':'offices'},axis=1).reset_index().npi.value_counts().head(87).index.to_list()

In [116]:
# Many seem to be part of hospital networks. But still, a conundrum regarding how to look at this geographically
# I guess if we are looking by region it's not too bad. I doubt many would have offices outside of the region?
df[df.npi.isin(multiple_counties)]

,medicaid_provider_id,npi,provider_or_facility_name,medicaid_type,profession_or_service,provider_specialty,service_address,city,state_x,zip_code,...,longitude,enrollment_begin_date,next_anticipated_revalidation_date,file_date,medically_fragile_children_directory_ind,provider_email,primary_taxonomy,selected_taxonomy,state_y,license_number
43,233550,1134359953,DEPAULA ROBERTO,FFS,PHYSICIAN,CHILD PSYCHIATRY,1249 5TH AVE,NEW YORK,NY,10029-4413,...,-73.95115,12/13/2013,07/01/2023,02/28/2022,N,NaN,Yes,2084P0804X - Psychiatry & Neurology Child & A...,NY,124580
44,233550,1134359953,DEPAULA ROBERTO,FFS,PHYSICIAN,CHILD PSYCHIATRY,157 E 72ND ST OFC H,NEW YORK,NY,10021-4333,...,-73.96171,12/13/2013,07/01/2023,02/28/2022,N,NaN,Yes,2084P0804X - Psychiatry & Neurology Child & A...,NY,124580
45,233550,1134359953,DEPAULA ROBERTO,FFS,PHYSICIAN,CHILD PSYCHIATRY,200 E 64TH ST,NEW YORK,NY,10065-7426,...,-73.96436,12/13/2013,07/01/2023,02/28/2022,N,NaN,Yes,2084P0804X - Psychiatry & Neurology Child & A...,NY,124580
46,233550,1134359953,DEPAULA ROBERTO,FFS,PHYSICIAN,CHILD PSYCHIATRY,3109 37TH ST,ASTORIA,NY,11103-3932,...,-73.91999,12/13/2013,07/01/2023,02/28/2022,N,NaN,Yes,2084P0804X - Psychiatry & Neurology Child & A...,NY,124580
47,233550,1134359953,DEPAULA ROBERTO,FFS,PHYSICIAN,CHILD PSYCHIATRY,344 FULTON AVE,HEMPSTEAD,NY,11550-3923,...,-73.62283,12/13/2013,07/01/2023,02/28/2022,N,NaN,Yes,2084P0804X - Psychiatry & Neurology Child & A...,NY,124580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17526,5536998,1184103970,AMPADU GIFTY GYEMFA,FFS,CLINICAL PSYCHOLOGIST,NaN,3317 ROCHAMBEAU AVE,BRONX,NY,10467-2846,...,-73.88059,01/15/2019,05/22/2024,02/28/2022,N,NaN,Yes,103TC2200X - Psychologist Clinical Child & Ad...,NY,022763
17527,5536998,1184103970,AMPADU GIFTY GYEMFA,FFS,CLINICAL PSYCHOLOGIST,NaN,3340 BAINBRIDGE AVE,BRONX,NY,10467-2802,...,-73.87914,01/15/2019,05/22/2024,02/28/2022,N,NaN,Yes,103TC2200X - Psychologist Clinical Child & Ad...,NY,022763
19006,6350774,1730575499,HUBERT MAYA FURMAN,FFS,PHYSICIAN,CHILD PSYCHIATRY,5141 BROADWAY,NEW YORK,NY,10034-1159,...,-73.91246,11/03/2020,03/03/2026,02/28/2022,N,NaN,Yes,2084P0804X - Psychiatry & Neurology Child & A...,NY,293045
19007,6350774,1730575499,HUBERT MAYA FURMAN,FFS,PHYSICIAN,CHILD PSYCHIATRY,55 PALMER AVE,BRONXVILLE,NY,10708-3403,...,-73.83724,11/03/2020,03/03/2026,02/28/2022,N,NaN,Yes,2084P0804X - Psychiatry & Neurology Child & A...,NY,293045


In [115]:
len(multiple_counties)

87

In [118]:
df.query("selected_taxonomy.str.contains(('child'), case=False, na=False)").drop_duplicates(subset='npi').to_csv('child-providers-1.csv',index=False)